<a href="https://colab.research.google.com/github/CarlosSotero/Analise_Estatistica_de_Salarios_em_Dados/blob/main/Detec%C3%A7%C3%A3o_EPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Instalando e importando bibliotecas**

In [1]:
# Instala o pacote "ultralytics" via pip.
# Esse pacote contém as implementações oficiais do YOLOv8, YOLOv9 e utilitários
# usados para detecção de objetos, segmentação, classificação e tracking.
!pip install ultralytics

# Atualizando a versão do Kaggle
!pip install --upgrade kagglehub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 20.6 MB/s eta 0:00:00
  Attempting uninstall: kagglehub
    Found existing installation: kagglehub 0.3.13
    Uninstalling kagglehub-0.3.13:
      Successfully uninstalled kagglehub-0.3.13


In [2]:
from ultralytics import YOLO          # Biblioteca da Ultralytics (YOLOv8/YOLOv9)
import kagglehub                      # Facilita baixar datasets do Kaggle programaticamente
import os                             # Funções de sistema de arquivos (criar pastas, checar caminhos, etc.)
import shutil                         # Operações de cópia/movimentação de diretórios/arquivos
from google.colab import drive        # Integração do Google Colab com o Google Drive

# Monta o Google Drive no ambiente do Colab em /content/drive
drive.mount('/content/drive')

# Caminho de destino no seu Drive onde o dataset será copiado
dir_path = "/content/drive/MyDrive/Colab Notebooks/Projeto Visão Computacional/HardHatDataset"

# Faz o download da versão mais recente do dataset "Hard Hat Detection" do Kaggle
# O kagglehub retorna o caminho local onde os arquivos foram baixados
path = kagglehub.dataset_download("andrewmvd/hard-hat-detection")

# Mostra o caminho local (cache) onde o dataset foi salvo pelo kagglehub
print("Caminho do dataset:", path)

# Garante que o diretório de destino exista (não dá erro se já existir)
os.makedirs(dir_path, exist_ok=True)

# Copia todo o conteúdo do dataset baixado (em 'path') para o diretório dentro do seu Drive
# dirs_exist_ok=True permite sobrescrever/mesclar se a pasta já existir (Python 3.8+)
shutil.copytree(path, dir_path, dirs_exist_ok=True)

# Confirmação para o usuário
print(f'Dataset copiado para: {dir_path}')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Mounted at /content/drive


100%|██████████| 1.22G/1.22G [00:59<00:00, 22.0MB/s]

Extracting files...


Caminho do dataset: /root/.cache/kagglehub/datasets/andrewmvd/hard-hat-detection/versions/1
Dataset copiado para: /content/drive/MyDrive/Colab Notebooks/Projeto Visão Computacional/HardHatDataset


## **Inspecionando o Dataset**

In [3]:
# Monta o caminho para a pasta que contém os arquivos de anotações (formato Pascal VOC em .xml)
anotacoes_path = os.path.join(dir_path, 'annotations')

# Monta o caminho para a pasta que contém as imagens
images_path = os.path.join(dir_path, 'images')

# Monta o caminho para a pasta que contém as anotações no formato YOLO: class_id x_center y_center width height
labels_path = os.path.join(dir_path, 'labels')
os.makedirs(labels_path, exist_ok=True)

# Módulo para manipular XML nativamente em Python
import xml.etree.ElementTree as ET

# Conjunto para guardar os nomes de classes únicos encontrados nas anotações
unique_classes = set()

# Percorre todos os arquivos dentro da pasta de anotações
for xml_file in os.listdir(anotacoes_path):
    # Faz o parse do arquivo XML atual (espera-se formato Pascal VOC)
    tree = ET.parse(os.path.join(anotacoes_path, xml_file))
    root = tree.getroot()

    # Em Pascal VOC, cada objeto anotado está dentro de uma tag <object>
    for obj in root.findall('object'):
        # Dentro de <object>, a tag <name> traz o nome da classe (ex.: 'hard-hat', 'person', etc.)
        unique_classes.add(obj.find('name').text)

# Exibe o conjunto de classes únicas encontradas nos XMLs
print(unique_classes)

{'helmet', 'head', 'person'}


In [4]:
# Classes que serão consideradas no processo de conversão (ordem define os IDs).
# Aqui: 'person' -> class_id 0; 'helmet' -> class_id 1.
classes =  ['person','helmet']

def convert_xml_to_yolo(xml_file):
    """
    Lê um arquivo XML no formato Pascal VOC e converte suas anotações
    para linhas no formato YOLO (class_id x_center y_center width height),
    com coordenadas normalizadas entre 0 e 1.
    """
    # Faz o parse do XML e obtém a raiz do documento.
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Lê dimensões da imagem (necessárias para normalizar as coordenadas).
    image_width = int(root.find('size/width').text)
    image_height = int(root.find('size/height').text)

    # Lista que armazenará as linhas no formato YOLO para este arquivo.
    yolo_lines = []

    # Para cada objeto anotado no XML...
    for obj in root.findall('object'):
        # Nome da classe anotada
        class_name = obj.find('name').text

        # Ignora classes que não estão na lista 'classes'
        # (ex.: se aparecer 'head', será pulado).
        if class_name not in classes:
            continue

        # Mapeia nome da classe -> índice (id da classe no YOLO)
        class_id = classes.index(class_name)

        # Lê a caixa delimitadora no formato Pascal VOC (xmin, ymin, xmax, ymax)
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        # Converte para o formato YOLO:
        # - centro (x, y) e (largura, altura), todos normalizados por (W, H).
        x_center = ((xmin + xmax) / 2) / image_width
        y_center = ((ymin + ymax) / 2) / image_height
        width = (xmax - xmin) / image_width
        height = (ymax - ymin) / image_height

        # Monta a linha no formato esperado pelo YOLO.
        yolo_lines.append(f'{class_id} {x_center} {y_center} {width} {height}')

    # Retorna todas as linhas YOLO para este XML (uma por objeto válido).
    return yolo_lines

# Converte todos os arquivos da pasta de anotações para o formato YOLO.
for xml_file in os.listdir(anotacoes_path):
    # Garante que só processe arquivos .xml
    if not xml_file.endswith(".xml"):
        continue

    # Caminho completo do XML atual
    xml_path = os.path.join(anotacoes_path, xml_file)

    # Gera as linhas no formato YOLO para este arquivo
    yolo_lines = convert_xml_to_yolo(xml_path)

    # Caminho do arquivo .txt de saída correspondente (mesmo nome, extensão .txt)
    label_file = os.path.join(labels_path, xml_file.replace(".xml", ".txt"))

    # Escreve o conteúdo no arquivo de rótulos (um objeto por linha)
    with open(label_file, "w") as f:
        f.write("\n".join(yolo_lines))

# Confirma no console quando todas as conversões terminarem
print("Conversão concluída.")

Conversão concluída.


In [5]:
# Lista os arquivos dentro de labels_path e mostra apenas os 5 primeiros.
os.listdir(labels_path)[:5]

# Abre o primeiro arquivo de rótulos (.txt) encontrado na pasta labels_path.
# Ele contém as anotações no formato YOLO: class_id x_center y_center width height
with open(os.path.join(labels_path, os.listdir(labels_path)[1])) as f:
    # Lê o conteúdo do arquivo e imprime no console.
    # Cada linha representa um objeto anotado em uma imagem.
    print(f'class_id x_center y_center width height:\n {f.read()}')


class_id x_center y_center width height:
 


**Contabilizando classes "person" e "helmet"**

In [6]:
count_person = 0
count_helmet = 0

for file in os.listdir(labels_path):
    with open(os.path.join(labels_path, file)) as f:
        for line in f.readlines():
            class_id = int(line.split()[0])
            if class_id == 0:
                count_person += 1
            elif class_id == 1:
                count_helmet += 1

if count_person != count_helmet:
  print('Dataset desbalanceado')

print(f'Person: {count_person}')
print(f'Helmet: {count_helmet}')

Dataset desbalanceado
Person: 751
Helmet: 18966


**Verificando a quantidade de arquivos vazios**

In [7]:
empty_files = 0

for file in os.listdir(labels_path):
  if os.path.getsize(os.path.join(labels_path,file)) == 0:
    empty_files += 1

print(f'Arquivos vazios: {empty_files}')

Arquivos vazios: 419


## **Divisão da Dataset em treino e teste**

In [8]:
# 'base_path' é o diretório raiz onde você colocou o dataset organizado
base_path = dir_path

# Caminho completo da pasta onde as imagens (JPEG/PNG) estão armazenadas
images_path = os.path.join(base_path, 'images')

# Caminho completo da pasta onde os arquivos YOLO (.txt) estão armazenados
labels_path = os.path.join(base_path, 'labels')

# Criação das pastas de split (separação do dataset)
# Estrutura padrão usada pelo YOLOv8:
#   images/train/
#   images/val/
#   labels/train/
#   labels/val/
for split in ['train', 'val']:
    # Cria a pasta de imagens do split atual ('train' ou 'val')
    os.makedirs(os.path.join(base_path, 'images', split), exist_ok=True)

    # Cria a pasta de labels do split atual ('train' ou 'val')
    os.makedirs(os.path.join(base_path, 'labels', split), exist_ok=True)

In [9]:
# Lista apenas os arquivos de imagem (.jpg) presentes na pasta de imagens.
# Isso ignora quaisquer outros tipos de arquivos.
image_files = [f for f in os.listdir(images_path) if f.endswith('.png')]

# Lista onde serão armazenados apenas os arquivos de imagem que possuem rótulos correspondentes.
valid_files = []

# Para cada imagem encontrada...
for img in image_files:
    # Nome esperado do arquivo de rótulo (.txt) correspondente.
    # Exemplo: "image_1.jpg" → "image_1.txt"
    label_file = img.replace('.png', '.txt')

    # Verifica se o arquivo de anotação correspondente existe na pasta labels_path.
    # Se existir, considera esta imagem como "válida".
    if os.path.exists(os.path.join(labels_path, label_file)):
        valid_files.append(img)

# Exibe a quantidade de imagens que possuem um arquivo de label correspondente.
print(f'Total imagens válidas: {len(valid_files)}')

Total imagens válidas: 5000


In [10]:
import random

# Define a semente para o gerador de números aleatórios
# Isso garante que o embaralhamento seja sempre o mesmo (reprodutibilidade)
random.seed(42)

# Embaralha a lista de arquivos válidos (valid_files)
random.shuffle(valid_files)

# Define o índice de corte para separar treino (80%) e validação (20%)
split_index = int(len(valid_files) * 0.8)

# Cria a lista de arquivos de treino (primeiros 80%)
train_files = valid_files[:split_index]

# Cria a lista de arquivos de validação (últimos 20%)
val_files = valid_files[split_index:]

# Imprime quantos arquivos ficaram em cada split
print("Train:", len(train_files))
print("Val:", len(val_files))

# Função para copiar arquivos de imagens e seus rótulos correspondentes
def copy_files(file_list, split):
    for img in file_list:
        # Para cada imagem, define o nome do arquivo de rótulo correspondente (.txt)
        label = img.replace(".png", ".txt")

        # Copia a imagem para a pasta correta (train ou val)
        shutil.copy(
            os.path.join(images_path, img),
            os.path.join(base_path, "images", split, img)
        )

        # Copia o rótulo para a pasta correta (train ou val)
        shutil.copy(
            os.path.join(labels_path, label),
            os.path.join(base_path, "labels", split, label)
        )

# Copia os arquivos de treino
copy_files(train_files, "train")

# Copia os arquivos de validação
copy_files(val_files, "val")

# Mensagem final confirmando que o split foi concluído
print("Split concluído com cópia.")

Train: 4000
Val: 1000
Split concluído com cópia.


## **Modelo YOLO**

In [11]:
model = YOLO("yolov8n.pt")

model.train(
    data='/content/drive/MyDrive/Colab Notebooks/Projeto Visão Computacional/data.yaml',
    epochs=50,
    imgsz=640,
    batch=16
)

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Colab Notebooks/Projeto Visão Computacional/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=au

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x785c954b0140>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804